In [1]:
# Workspace Configuration Details
from azureml.core import Workspace, Experiment
from azureml.core import Workspace

ws = Workspace.from_config()
ws.write_config(path='.azureml')

experiment_name = 'udacity_project_solution_real'
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,  
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133262
Azure region: southcentralus
Resource group: aml-quickstarts-133262


In [2]:
# Creating CPU cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "optimizeml"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cpu_cluster = ComputeTarget(workspace=ws,name=cpu_cluster_name)
    print("CPU Cluster exist")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
# to check basic info and parameters
# from azureml.train.sklearn import SKLearn
# SKLearn?
# from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# HyperDriveConfig?

## HyperDrive Run

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, choice
import os
import shutil   # imported to copy the train script


# Specify parameter sampler
ps = RandomParameterSampling( 
    {'--C' : choice(1,2,3,4,5),
     '--max_iter' : choice(100,150,200,250)}
)

# Specify a Policy

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_folder,compute_target=cpu_cluster,entry_script='train.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
experiment = Experiment(workspace=ws, name='HyperDriveMLOptimize')
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

### Retrieving best HyperDrive Run Model

In [7]:
import joblib
# Get your best run and save the model from that run.
# best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '2', '--max_iter', '150']


In [8]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_f92d0eb98e6b036f981093c56b54cc3ed6b10a039f20ccd5c70ec8f3ffd86ba2_d.txt', 'azureml-logs/65_job_prep-tvmps_f92d0eb98e6b036f981093c56b54cc3ed6b10a039f20ccd5c70ec8f3ffd86ba2_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_f92d0eb98e6b036f981093c56b54cc3ed6b10a039f20ccd5c70ec8f3ffd86ba2_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/104_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [9]:
#best_run.register_model?

In [10]:
# Registering best Model HyperDrive
model = best_run.register_model(model_name='logistic_hyperdrive', model_path='outputs/model.joblib')

In [11]:
best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

### YOUR CODE HERE ###

Best Run Id:  HD_5e6a7c9d-478f-4516-8b37-9de7a5356dfc_4

 Accuracy: 0.9120890237733941


## AutoML Run

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

auto_loc = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ads = TabularDatasetFactory.from_delimited_files(auto_loc, separator=',') ### YOUR CODE HERE ###


In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ads)
data = x.join(y)      # joined the output label
data.to_csv('data.csv')

In [14]:
from azureml.core import Dataset
data_auto = Dataset.get_by_name(ws, name='Bank-marketing')

In [41]:
# from azure.core import Dataset
# datastore = ws.get_default_datastore()
# datastore.upload(src_dir='.',show_progress=True,)
#type(data['age'])

pandas.core.series.Series

In [28]:
# from azureml.train.automl.utilities import get_primary_metrics 
# get_primary_metrics('classification')

['precision_score_weighted',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'AUC_weighted',
 'accuracy']

In [6]:
# from azureml.train.automl import AutoMLConfig

# AutoMLConfig?

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_auto,
    label_column_name='y',
    compute_target=cpu_cluster,
    enable_voting_ensemble=False,
    enable_stack_ensemble=False,
    enable_onnx_compatible_models=True,
    n_cross_validations=3)

In [16]:
from azureml.core.experiment import Experiment
# Choose a name for the experiment.
experiment_name = 'logistic-automl'

experiment = Experiment(ws, experiment_name)

In [17]:
# Submit your automl run
run = experiment.submit(automl_config, show_output=True)
### YOUR CODE HERE ###

Running on remote.
No run_configuration provided, running on optimizeml with default configuration
Running on remote compute: optimizeml
Parent Run ID: AutoML_de41f7c9-392e-454f-9de0-cd82abb14a14

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards

In [18]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Retrieving Best AutoML Model

In [21]:
# Retrieve and save your best automl model.
automl_best_run, fitted_model = run.get_output()
#print(automl_best_run)
print(fitted_model)
### YOUR CODE HERE ###

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('SparseNormalizer',
                 <azureml.auto...
                                   colsample_bytree=0.5, eta=0.001, gamma=0.01,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=5, max_leaves=3,
                                   min_child_weight=1, missing=nan,
                                   n_estimators=50, n_jobs=1, nthread=None,
                

In [22]:
# Registering AutoML model
automl_best_run.register_model(model_name='classification_automl',model_path='outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-133262', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-133262'), name=classification_automl, id=classification_automl:2, version=2, tags={}, properties={})

In [23]:
# automl metrics
automl_metrics = automl_best_run.get_metrics()
for metric in automl_metrics:
    print('Metric name : {} = {}'.format(metric,automl_metrics[metric]))

Metric name : balanced_accuracy = 0.7410642199020788
Metric name : accuracy = 0.9152958631226342
Metric name : log_loss = 0.17967058371089908
Metric name : f1_score_weighted = 0.9108460245453491
Metric name : weighted_accuracy = 0.9586048660964389
Metric name : f1_score_macro = 0.7651573616009838
Metric name : matthews_correlation = 0.5362710889024622
Metric name : AUC_micro = 0.9802582233464068
Metric name : precision_score_weighted = 0.9087795440617022
Metric name : precision_score_macro = 0.7985796763051732
Metric name : recall_score_weighted = 0.9152958631226342
Metric name : norm_macro_recall = 0.4821284398041576
Metric name : precision_score_micro = 0.9152958631226342
Metric name : recall_score_micro = 0.9152958631226342
Metric name : f1_score_micro = 0.9152958631226343
Metric name : average_precision_score_micro = 0.9810290140806962
Metric name : AUC_macro = 0.9456440257431185
Metric name : average_precision_score_weighted = 0.9547021121099214
Metric name : recall_score_macro = 

In [24]:
# Hyperdrive metrics
hyperdrive_metrics = best_run.get_metrics()
for metric in hyperdrive_metrics:
    print('Metric name : {} = {}'.format(metric,hyperdrive_metrics[metric]))

Metric name : Regularization Strength: = 2.0
Metric name : Max iterations: = 150
Metric name : Accuracy = 0.9120890237733941


In [25]:
# parameter_values = best_run.get_details()
# parameter_values

In [26]:
# Deleting cluster
cpu_cluster.delete()